In [1]:
#Removes 2 O atoms and 2 H atoms

import random
import re

input_file = 'ada_1.pdb'
output_file = 'ada_2.pdb'

# Open the PDB file
with open(input_file, 'r') as pdb_file:
    # Initialize lists to store the indices of oxygen, hydrogen, and carbon atoms
    oxygen_indices = []
    hydrogen_indices = []
    carbon_indices = []

    # Iterate through the lines of the file
    for line in pdb_file:
        if line.startswith('ATOM'):
            atom_type = line.split()[2]
            atom_index = int(line.split()[1])
            

            if 'O' in atom_type:
                oxygen_indices.append(atom_index)
            elif 'H' in atom_type:
                hydrogen_indices.append(atom_index)
            elif 'C' in atom_type:
                carbon_indices.append(atom_index)

# Generate a list of indices to remove from the oxygen indices list
indices_to_remove = []
for i in range(0, len(oxygen_indices), 3):
    group_indices = oxygen_indices[i:i+3]
    if len(group_indices) >= 2:
        remove_indices = random.sample(group_indices, 2)
        indices_to_remove.extend(remove_indices)

# Remove the selected indices from the oxygen indices list
oxygen_indices = [index for index in oxygen_indices if index not in indices_to_remove]

# Generate a list of indices to remove from the hydrogen indices list
hydrogen_to_remove = []
for oxygen_index in oxygen_indices:
    # Calculate the corresponding hydrogen indices to remove
    hydrogen_to_remove.extend([oxygen_index + 1, oxygen_index + 2])

# Remove the selected hydrogen indices from the hydrogen indices list
hydrogen_indices = [index for index in hydrogen_indices if index not in hydrogen_to_remove]

#########################

# Open the PDB file again to re-enumerated
with open(input_file, 'r') as pdb_file:
    # Initialize a list to store the remaining rows
    remaining_rows = []
    line_count = 0
    

    # Iterate through the lines of the file
    for line in pdb_file:
        if line.startswith('ATOM'):
            atom_type = line.split()[2]
            atom_index = int(line.split()[1])


            if 'O' in atom_type and atom_index in oxygen_indices:
                line_count += 1
                new_line = line[:6] + str(line_count).rjust(5) + line[11:]
                remaining_rows.append(new_line)
            elif 'H' in atom_type and atom_index in hydrogen_indices:
                line_count += 1
                new_line = line[:6] + str(line_count).rjust(5) + line[11:]
                remaining_rows.append(new_line)
            elif 'C' in atom_type and atom_index in carbon_indices:
                line_count += 1
                new_line = line[:6] + str(line_count).rjust(5) + line[11:]
                remaining_rows.append(new_line)

        elif not line.startswith('CONECT'):
            remaining_rows.append(line)

# Write the remaining rows with re-enumerated lines to a new PDB file
with open(output_file, 'w') as output_file:
    output_file.writelines(remaining_rows)


In [ ]:
#Enumerate in Groups

input_file = 'ada_2.pdb'
output_file = 'ada_3.pdb'

# Open the PDB file again to re-enumerated
with open(input_file, 'r') as pdb_file:
    # Initialize a list to store the remaining rows
    remaining_rows = []
    line_count = 0
    # Initialize lists to store the indices of oxygen, hydrogen, and carbon atoms
    oxygen_indices = []
    hydrogen_indices = []
    carbon_indices = []
          

    # Iterate through the lines of the file
    for line in pdb_file:
        if line.startswith('ATOM'):
            atom_type = line.split()[2]
            atom_index = int(line.split()[1])

            line_count += 1
            new_line = line[:6] + str(line_count).rjust(5) + line[11:]
            remaining_rows.append(new_line)

        elif not line.startswith('CONECT'):
            remaining_rows.append(line)
            
        if line_count == 25:
            line_count = 0
            


# Write the remaining rows with re-enumerated lines to a new PDB file
with open(output_file, 'w') as output_file:
    output_file.writelines(remaining_rows)

In [11]:
# Re-order the atoms

def read_pdb_in_groups(file_path, group_size):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    non_atom_lines = []
    current_group = []
    groups = []

    for line in lines:
        if line.startswith("ATOM"):
            current_group.append(line)

            # Check if the current group is complete
            if len(current_group) == group_size:
                # Find the indices of the oxygen atom (O) and the atom at the second position
                oxygen_index = None
                atom2_index = None
                for i, atom_line in enumerate(current_group):
                    atom_type = atom_line[76:78].strip()
                    if atom_type == "O":
                        oxygen_index = i
                    elif i == 1:  # Only consider the atom at the second position
                        atom2_index = i

                # Swap the lines of the oxygen atom (row 1) and the atom at row position 2
                if oxygen_index is not None and atom2_index is not None:
                    current_group[oxygen_index], current_group[atom2_index] = current_group[atom2_index], current_group[oxygen_index]
                #print(oxygen_index)  # Add this line to check the value
                if oxygen_index == 7:
                    current_group[2], current_group[3] = current_group[3], current_group[2]
                    current_group[4], current_group[7] = current_group[7], current_group[4]
                    current_group[3], current_group[4] = current_group[4], current_group[3]
                    current_group[5], current_group[6] = current_group[6], current_group[5]
                    current_group[6], current_group[7] = current_group[7], current_group[6]
                    
                #if oxygen_index == 1:
                    #current_group[2], current_group[3] = current_group[3], current_group[2]
                if oxygen_index == 4:
                    current_group[2], current_group[3] = current_group[3], current_group[2]
                    current_group[3], current_group[4] = current_group[4], current_group[3]


                groups.append(current_group)
                current_group = []

    # Add the last group if it's not complete
    if current_group:
        groups.append(current_group)

    return non_atom_lines, groups
     

def write_lines_to_file(file_path, lines):
    with open(file_path, 'w') as file:
        for line in lines:
            file.write(line)

# Usage example
pdb_file = "ada_3.pdb"
group_size = 25

non_atom_lines, pdb_groups = read_pdb_in_groups(pdb_file, group_size)

# Write all lines to a new file
output_file = "ada_4.pdb"
with open(output_file, 'w') as file:
    for line in non_atom_lines:
        file.write(line)
    for group in pdb_groups:
        for line in group:
            file.write(line)


In [12]:
# Open the PDB file again to re-enumerate and rearrange rows
output_file = 'ada_4.pdb'

with open(output_file, 'r') as pdb_file:
    # Initialize lists to store the remaining rows and the MASTER/END rows
    remaining_rows = []
    master_end_rows = []

    # Iterate through the lines of the file
    for line in pdb_file:
        if line.startswith('MASTER') or line.startswith('END'):
            master_end_rows.append(line)
        else:
            remaining_rows.append(line)

# Combine the remaining rows and the MASTER/END rows
remaining_rows.extend(master_end_rows)

# Write the remaining rows with re-enumerated lines and the MASTER/END rows to a new PDB file
reordered_output_file = "ada_5.pdb"
with open(reordered_output_file, 'w') as output_file:
    output_file.writelines(remaining_rows)


In [14]:
# Open the PDB file again to re-enumerate
input_file = 'ada_5.pdb'  # Replace with the actual input file path or filename
oxygen_indices = []  # Replace with the actual oxygen atom indices to re-enumerate
hydrogen_indices = []  # Replace with the actual hydrogen atom indices to re-enumerate
carbon_indices = []  # Replace with the actual carbon atom indices to re-enumerate

with open(input_file, 'r') as pdb_file:
    # Initialize a list to store the remaining rows
    remaining_rows = []
    line_count = 0

    # Initialize lists to store the indices of oxygen, hydrogen, and carbon atoms
    oxygen_indices = []
    hydrogen_indices = []
    carbon_indices = []

    # Iterate through the lines of the file
    for line in pdb_file:
        if line.startswith('ATOM'):
            atom_type = line.split()[2]
            atom_index = int(line.split()[1])
            

            if 'O' in atom_type:
                oxygen_indices.append(atom_index)
            elif 'H' in atom_type:
                hydrogen_indices.append(atom_index)
            elif 'C' in atom_type:
                carbon_indices.append(atom_index)

            if 'O' in atom_type and atom_index in oxygen_indices:
                line_count += 1
                new_line = line[:6] + str(line_count).rjust(5) + line[11:]
                remaining_rows.append(new_line)
            elif 'H' in atom_type and atom_index in hydrogen_indices:
                line_count += 1
                new_line = line[:6] + str(line_count).rjust(5) + line[11:]
                remaining_rows.append(new_line)
            elif 'C' in atom_type and atom_index in carbon_indices:
                line_count += 1
                new_line = line[:6] + str(line_count).rjust(5) + line[11:]
                remaining_rows.append(new_line)

        elif not line.startswith('CONECT'):
            remaining_rows.append(line)

# Write the remaining rows with re-enumerated lines to a new PDB file
output_file = 'ada.pdb'  # Replace with the desired output file path or filename
with open(output_file, 'w') as output_file:
    output_file.writelines(remaining_rows)
